In [ ]:
import anvil.server

anvil.server.connect("anvil key")

In [44]:
#PreProcess Part
import numpy as np
import matplotlib.pyplot as plt
import cv2

def borders(here_img, thresh):
    size = here_img.shape
    check = int(115 * size[0] / 600)
    image = here_img[:]
    top, bottom = 0, size[0] - 1
    shape = size

    #find the background color for empty column
    bg = np.repeat(thresh, shape[1])
    count = 0
    for row in range(1, shape[0]):
        if  (np.equal(bg, image[row]).any()) == True:
            count += 1
        else:
            count = 0
        if count >= check:
            top = row - check
            break
    
    
    shape = image.shape
    bg = np.repeat(thresh, shape[1])
    count = 0
    rows = np.arange(1, shape[0])
    for row in rows[::-1]:
        if  (np.equal(bg, image[row]).any()) == True:
            count += 1
        else:
            count = 0
        if count >= check:
            bottom = row + count
            break
    
    d1 = (top - 2) >= 0 
    d2 = (bottom + 2) < size[0]
    d = d1 and d2
    if(d):
        b = 2
    else:
        b = 0
    
    return (top, bottom, b)

def detect_text(main_image, gray_img, localized, bc):        
        cimg = cv2.resize(localized, (30, 30))
        bordersize = 1
        nimg = cv2.copyMakeBorder(cimg, top=bordersize, bottom=bordersize, left=bordersize, right=bordersize, borderType=cv2.BORDER_CONSTANT, value=[255-bc, 0, 0])

        return main_image, nimg

def segmentation(bordered, thresh):
    try:
        shape = bordered.shape
        check = int(50 * shape[0] / 320)
        image = bordered[:]
        image = image[check:].T
        shape = image.shape

        #find the background color for empty column
        bg = np.repeat(255 - thresh, shape[1])
        bg_keys = []
        for row in range(1, shape[0]):
            if  (np.equal(bg, image[row]).all()):
                bg_keys.append(row)            

        lenkeys = len(bg_keys)-1
        new_keys = [bg_keys[1], bg_keys[-1]]
        for i in range(1, lenkeys):
            if (bg_keys[i+1] - bg_keys[i]) > check:
                new_keys.append(bg_keys[i])

        new_keys = sorted(new_keys)
        segmented_templates = []
        first = 0
        for key in new_keys[1:]:
            segment = bordered.T[first:key]
            segmented_templates.append(segment.T)
            first = key
        last_segment = bordered.T[new_keys[-1]:]
        segmented_templates.append(last_segment.T)
        
        #check if each segment shape is enough to do recognition
        return(segmented_templates)
    except:
        return [bordered]

def localize(main_image, gray_img, localized, bc, show):
    #open the template as gray scale image
        template = localized
        #print(template.shape)
        width, height = template.shape[::-1] #get the width and height
        #match the template using cv2.matchTemplate
        match = cv2.matchTemplate(gray_img, template, cv2.TM_CCOEFF_NORMED)
        threshold = 0.8
        position = np.where(match >= threshold) #get the location of template in the image
        for point in zip(*position[::-1]): #draw the rectangle around the matched template
            cv2.rectangle(main_image, point, (point[0] + width, point[1] + height), (255 - bc, 0, bc ), 2)
        return main_image

def preprocess(bgr_img):#gray image   
    img = bgr_img[:]   
    blur = cv2.GaussianBlur(img,(5,5),0)
    ret,th_img = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU) #converts black to white and inverse
    rows, cols = th_img.shape
    bg_test = np.array([th_img[i][i] for i in range(5)])
    if bg_test.all() == 0:
        text_color = 255
    else:
        text_color = 0

    tb = borders(th_img, text_color)
    lr = borders(th_img.T, text_color)
    dummy = int(np.average((tb[2], lr[2]))) + 2
    template = th_img[tb[0]+dummy:tb[1]-dummy, lr[0]+dummy:lr[1]-dummy]
    
    plt.imshow(template)
    plt.show()
    
    segments = segmentation(template, text_color)
    
    return segments, template, th_img, text_color    

In [45]:
#Prediction Part
import numpy as np
from keras.models import model_from_json
from keras.models import load_model

def prediction(img):
   # load json and create model
    json_file = open('model\cnn.json', 'r')
    
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights("model\cnn.h5")
    #print("Loaded model from disk")
    
    loaded_model.save('cnn.hdf5')
    #loaded_model=load_model('D:\ATIT\Minor Project\Jupyter Project\Minor Project\cnn2\cnn2.h5')
    loaded_model=load_model('cnn.hdf5', compile=False)
    
    characters = '०,१,२,३,४,५,६,७,८,९,क,ख,ग,घ,ङ,च,छ,ज,झ,ञ,ट,ठ,ड,ढ,ण,त,थ,द,ध,न,प,फ,ब,भ,म,य,र,ल,व,श,ष,स,ह,क्ष,त्र,ज्ञ'
    characters = characters.split(',')
    
    x = np.asarray(img, dtype = np.float32).reshape(1, 32, 32, 1) / 255 
    
    output = loaded_model.predict(x)
    output = output.reshape(46)
    predicted = np.argmax(output)
    devanagari_label = characters[predicted]
    success = output[predicted] * 100
    
    return devanagari_label, success


In [46]:
#translate to  english 

import requests
@anvil.server.callable
def translate(text):
    url = "https://google-translate1.p.rapidapi.com/language/translate/v2"
    payload = 'q=%s&source=ne&target=en' % text
    headers = {
    'content-type': "application/x-www-form-urlencoded",
    'accept-encoding': "application/gzip",
    'x-rapidapi-key': "96d9f48971msh84b6d75df50fc0cp18c433jsn93d71fee7939",
    'x-rapidapi-host': "google-translate1.p.rapidapi.com"
    }

    response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
    translatedText = response.json()['data']['translations'][0]['translatedText']
    return translatedText

In [6]:
#Recognition part and Website

import numpy as np
import anvil.media 
import os
import cv2

@anvil.server.callable
def recognition_new(gray_image, show):
    with anvil.media.TempFile(gray_image) as filename:
        gray_image = cv2.imread(filename,0)
        segments, template, th_img, text_color = preprocess(gray_image)
        labels = []
        accuracy = []
        show_img = gray_image[:]

        for segment in segments: 
            plt.imshow(segment)
            plt.show()
            recimg, bimg = detect_text(show_img, th_img, segment, text_color)
            label, sure = prediction(bimg)
            if(sure > 75):
                labels.append(str(label))
                accuracy.append(sure)
                show_img = localize(show_img, th_img, segment, text_color, show)
            char = labels
        accuracy = np.average(accuracy)
        char = ''.join(char)
        if accuracy < 75:
            recimg, bimg = detect_text(show_img, th_img, template, text_color)
            show_img = localize(show_img, th_img, template, text_color, show)
            char, accuracy = prediction(bimg)

        if (show == 'show'):
            plt.imshow(show_img)
            plt.title('Detecting')
            plt.xticks([])
            plt.yticks([])
            plt.show()
        else:
            cv2.imshow('Detecting..', cv2.cvtColor(show_img, cv2.COLOR_GRAY2BGR))

        print('Word : ', char,' Avg Accuracy : ',"%.2f" % round(accuracy,2), '%')
        return char, accuracy